<a href="https://colab.research.google.com/github/slisovych/machine_learning_hometasks/blob/main/HW_2_4_kNN_%D0%9A%D1%80%D0%BE%D1%81%D0%B2%D0%B0%D0%BB%D1%96%D0%B4%D0%B0%D1%86%D1%96%D1%8F_%D1%96_%D1%82%D1%8E%D0%BD%D0%B8%D0%BD%D0%B3_%D0%B3%D1%96%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D1%96%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [1]:
!pip install scikit-learn --upgrade

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from process_bank_churn_4 import preprocess_data

In [4]:
df = pd.read_csv("train.csv")

X_train, train_targets, X_val, val_targets, input_cols, scaler, encoder = preprocess_data(df)

In [5]:
X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
7180,0.599045,0.214286,0.1,0.626382,0.000000,1.0,1.0,0.719772,1.0,0.0,0.0,0.0,1.0
10393,0.603819,0.375000,0.2,0.848836,0.000000,1.0,0.0,0.727603,1.0,0.0,0.0,1.0,0.0
80,0.653938,0.303571,0.6,0.554522,0.333333,1.0,0.0,0.872180,0.0,1.0,0.0,0.0,1.0
3365,0.568019,0.714286,0.0,0.000000,0.333333,0.0,1.0,0.257797,0.0,0.0,1.0,0.0,1.0
12236,0.658711,0.053571,0.3,0.000000,0.333333,1.0,1.0,0.742837,1.0,0.0,0.0,0.0,1.0


1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

In [7]:
knn = KNeighborsClassifier()
knn.fit(X_train, train_targets)

# передбачення ймовірностей
train_probs = knn.predict_proba(X_train)[:, 1]
val_probs = knn.predict_proba(X_val)[:, 1]

# AUROC
train_auroc = roc_auc_score(train_targets, train_probs)
val_auroc = roc_auc_score(val_targets, val_probs)

train_auroc, val_auroc

(0.9559428457733543, 0.852585568283147)

Високий AUROC на тренувальному наборі означає, що модель добре навчається на доступних даних і має високу здатність розрізняти класи.

High variance: значне зниження AUROC на валідаційних даних означає,
що модель погано узагальнює знання і може бути перенавчена.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [8]:
from sklearn.model_selection import GridSearchCV

# сітка параметрів для пошуку
param_grid = {'n_neighbors': list(range(1, 21))}

# пошук найкращого значення n_neighbors
search = GridSearchCV(knn, param_grid, cv=5, scoring='roc_auc', verbose=1)
search.fit(X_train, train_targets)

# найкраще значення n_neighbors та найкраща модель
best_n_neighbors = search.best_params_['n_neighbors']
knn_best = search.best_estimator_

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [9]:
knn_best

KNeighborsClassifier(n_neighbors=19)

In [10]:
train_probs_best = knn_best.predict_proba(X_train)[:, 1]
val_probs_best = knn_best.predict_proba(X_val)[:, 1]

train_auroc_best = roc_auc_score(train_targets, train_probs_best)
val_auroc_best = roc_auc_score(val_targets, val_probs_best)

best_n_neighbors, train_auroc_best, val_auroc_best


(19, 0.9227310044165035, 0.8908141847863366)

На валідаційній вибірці якість покращилася:

AUROC на валідації зріс з 0.8525 до 0.8908 – це означає, що модель краще узагальнює дані.

Проте точність на тренувальному наборі знизилася:

AUROC на тренуванні впав з 0.9559 до 0.9227, що свідчить про менше перенавчання (overfitting).

У попередньому ДЗ Дерево прийняття рішень значення AUCROC приблизно таке саме на тренувальних даних (0.9287), проте на валідаційних результат крещий (0.9225).

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [11]:
import time
from sklearn.tree import DecisionTreeClassifier

# сітка параметрів
param_grid_dt = {
    'max_depth': list(range(1, 21, 2)),  # від 1 до 20 з кроком 2
    'max_leaf_nodes': list(range(2, 11, 1))  # від 2 до 10 з кроком 1
}

dt = DecisionTreeClassifier(random_state=42)

# час початку
start_time = time.time()

search_dt = GridSearchCV(dt, param_grid_dt, cv=3, scoring='roc_auc', verbose=1)
search_dt.fit(X_train, train_targets)

# час завершення
end_time = time.time()
search_time = end_time - start_time

# найкращі значення параметрів та модель
best_params_dt = search_dt.best_params_
dt_best = search_dt.best_estimator_

train_probs_dt = dt_best.predict_proba(X_train)[:, 1]
val_probs_dt = dt_best.predict_proba(X_val)[:, 1]

train_auroc_dt = roc_auc_score(train_targets, train_probs_dt)
val_auroc_dt = roc_auc_score(val_targets, val_probs_dt)

best_params_dt, train_auroc_dt, val_auroc_dt, search_time


Fitting 3 folds for each of 90 candidates, totalling 270 fits


({'max_depth': 5, 'max_leaf_nodes': 10},
 0.9014754782174744,
 0.9002184649152891,
 3.924344301223755)

Оптимізоване Decision Tree є кращою моделлю у порівнянні з початковим kNN. Проте це гірший результат у порівнянні з Decision Tree  з попереднього ДЗ:

Train ROC-AUC: 0.9287

Validation ROC-AUC: 0.9225

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [37]:
from sklearn.model_selection import RandomizedSearchCV

params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

dt = DecisionTreeClassifier(class_weight="balanced", random_state=42)

# час початку
start_time = time.time()

# RandomizedSearchCV з 40 ітераціями
random_search_dt = RandomizedSearchCV(
    dt, param_distributions=params_dt, n_iter=40, cv=3, scoring='roc_auc',
    verbose=1, random_state=42
)
random_search_dt.fit(X_train, train_targets)

# час завершення
end_time = time.time()
search_time_random = end_time - start_time

# найкращі значення параметрів та модель
best_params_dt_random = random_search_dt.best_params_
dt_random_search_best = random_search_dt.best_estimator_

train_probs_dt_random = dt_random_search_best.predict_proba(X_train)[:, 1]
val_probs_dt_random = dt_random_search_best.predict_proba(X_val)[:, 1]

train_auroc_dt_random = roc_auc_score(train_targets, train_probs_dt_random)
val_auroc_dt_random = roc_auc_score(val_targets, val_probs_dt_random)

best_params_dt_random, train_auroc_dt_random, val_auroc_dt_random, search_time_random


Fitting 3 folds for each of 40 candidates, totalling 120 fits


({'splitter': 'best',
  'min_samples_split': 10,
  'min_samples_leaf': 15,
  'min_impurity_decrease': 0.0001,
  'max_leaf_nodes': 14,
  'max_depth': 6,
  'criterion': 'entropy'},
 0.9173500456457142,
 0.9155583373345223,
 2.561749219894409)

Модель із RandomizedSearchCV є кращою за GridSearchCV!

Вища якість на валідаційних даних (0.9155 vs. 0.9002).

Більш глибоке дерево (max_depth=16) краще розпізнає складні закономірності.

Швидший пошук гіперпараметрів.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [24]:
from process_bank_churn_4 import preprocess_new_data

In [14]:
test_raw_df = pd.read_csv('test.csv')

In [25]:
print("Test dataset columns before encoding:", test_raw_df.columns)
preprocessed_test = preprocess_new_data(test_raw_df, input_cols, scaler, encoder, scale_numeric=False)
print("Test dataset columns after preprocessing:", preprocessed_test.columns)


Test dataset columns before encoding: Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')
Test dataset columns after preprocessing: Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')


In [26]:
preprocessed_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,584.0,30.0,2.0,146053.66,1.0,1.0,1.0,157891.86,0.0,1.0,0.0,0.0,1.0
1,551.0,39.0,5.0,0.00,2.0,1.0,1.0,67431.28,1.0,0.0,0.0,0.0,1.0
2,706.0,43.0,8.0,0.00,2.0,1.0,0.0,156768.45,1.0,0.0,0.0,0.0,1.0
3,717.0,45.0,3.0,0.00,1.0,1.0,1.0,166909.87,0.0,0.0,1.0,0.0,1.0
4,592.0,43.0,8.0,0.00,2.0,1.0,1.0,143681.97,0.0,0.0,1.0,0.0,1.0


In [27]:
submission = pd.read_csv('sample_submission.csv')

In [28]:
submission.head()

,id,Exited
0,15000,0.5
1,15001,0.5
2,15002,0.5
3,15003,0.5
4,15004,0.5


In [38]:
proba_predictions = dt_random_search_best.predict_proba(preprocessed_test)[:, 1]

In [39]:
submission['Exited'] = proba_predictions

In [40]:
submission['Exited']


,Exited
0,0.998354
1,0.998354
2,0.998354
3,0.998354
4,0.998354
...,...
9995,0.998354
9996,0.998354
9997,0.998354
9998,0.998354


In [22]:
submission.to_csv('submission_knn.csv', index=False)

In [34]:
print(preprocessed_test.nunique())

CreditScore           365
Age                    56
Tenure                 12
Balance              2514
NumOfProducts           4
HasCrCard               2
IsActiveMember          2
EstimatedSalary      4980
Geography_France        2
Geography_Germany       2
Geography_Spain         2
Gender_Female           2
Gender_Male             2
dtype: int64


In [35]:
print("Train columns:", X_train.columns.tolist())
print("Test columns:", preprocessed_test.columns.tolist())

Train columns: ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France', 'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male']
Test columns: ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France', 'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male']


In [42]:
test_probs_dt = dt_random_search_best.predict_proba(preprocessed_test)[:, 1]
print(pd.Series(test_probs_dt).describe())

count    1.000000e+04
mean     9.983544e-01
std      2.413746e-13
min      9.983544e-01
25%      9.983544e-01
50%      9.983544e-01
75%      9.983544e-01
max      9.983544e-01
dtype: float64
